# Credit Card Fraud Detection - EDA and Baseline Model

이 노트북에서는 신용카드 사기 탐지 데이터를 탐색하고 기본 모델을 구축합니다.

## 목차
1. 데이터 로딩 및 기본 탐색
2. EDA (Exploratory Data Analysis)
3. 데이터 분할 (Train/Validation/Test)
4. Feature Engineering
5. Baseline Logistic Regression 모델
6. 모델 평가 (불균형 데이터 고려)
7. 결과 분석 및 개선 방향

## 1. 데이터 로딩 및 기본 탐색

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report, roc_auc_score, roc_curve
)

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
# 데이터 로딩
# 주의: 데이터를 먼저 다운로드해야 합니다 (README.md 참고)
df = pd.read_csv('../data/creditcard.csv')
print(f"데이터 크기: {df.shape}")
df.head()

In [ ]:
# 데이터 기본 정보
df.info()

In [ ]:
# 기본 통계
df.describe()

In [ ]:
# 결측치 확인
print("결측치:")
print(df.isnull().sum())

## 2. EDA - Target Variable 분석

신용카드 사기 탐지의 가장 큰 특징은 **클래스 불균형**입니다.

In [ ]:
# Class 분포 확인
print("Class 분포:")
print(df['Class'].value_counts())
print("\nClass 비율:")
print(df['Class'].value_counts(normalize=True))
print(f"\n사기 비율: {df['Class'].mean():.4f} ({df['Class'].mean()*100:.2f}%)")

In [ ]:
# Class 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Count plot
df['Class'].value_counts().plot(kind='bar', ax=axes[0])
axes[0].set_title('Class Distribution (Count)')
axes[0].set_xlabel('Class (0: Normal, 1: Fraud)')
axes[0].set_ylabel('Count')

# Pie chart
df['Class'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.2f%%')
axes[1].set_title('Class Distribution (Percentage)')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

### Amount와 Time 특성 분석

In [ ]:
# Amount 분포 비교 (정상 vs 사기)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Amount distribution
df[df['Class'] == 0]['Amount'].plot(kind='hist', bins=50, ax=axes[0], alpha=0.7, label='Normal')
df[df['Class'] == 1]['Amount'].plot(kind='hist', bins=50, ax=axes[0], alpha=0.7, label='Fraud')
axes[0].set_title('Amount Distribution by Class')
axes[0].set_xlabel('Amount')
axes[0].legend()

# Box plot
df.boxplot(column='Amount', by='Class', ax=axes[1])
axes[1].set_title('Amount by Class (Boxplot)')
axes[1].set_xlabel('Class (0: Normal, 1: Fraud)')

plt.tight_layout()
plt.show()

In [ ]:
# Amount 통계 비교
print("Amount 통계 (정상 거래):")
print(df[df['Class'] == 0]['Amount'].describe())
print("\nAmount 통계 (사기 거래):")
print(df[df['Class'] == 1]['Amount'].describe())

In [ ]:
# Time 분포 분석
fig, ax = plt.subplots(figsize=(12, 4))
df[df['Class'] == 0]['Time'].plot(kind='hist', bins=50, alpha=0.7, label='Normal', ax=ax)
df[df['Class'] == 1]['Time'].plot(kind='hist', bins=50, alpha=0.7, label='Fraud', ax=ax)
ax.set_title('Time Distribution by Class')
ax.set_xlabel('Time (seconds)')
ax.legend()
plt.show()

### V1-V28 특성 분석 (샘플)

In [ ]:
# V1-V5 특성의 분포 비교 (샘플)
v_features = ['V1', 'V2', 'V3', 'V4', 'V5']

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

for i, feature in enumerate(v_features):
    df[df['Class'] == 0][feature].plot(kind='hist', bins=50, alpha=0.7, label='Normal', ax=axes[i])
    df[df['Class'] == 1][feature].plot(kind='hist', bins=50, alpha=0.7, label='Fraud', ax=axes[i])
    axes[i].set_title(f'{feature} Distribution')
    axes[i].legend()

axes[-1].axis('off')  # 마지막 subplot 숨기기
plt.tight_layout()
plt.show()

## 3. 데이터 분할 (Train/Validation/Test)

불균형 데이터이므로 stratify를 사용하여 각 세트에 동일한 비율로 사기 거래가 포함되도록 합니다.

In [ ]:
# Train/Test split (80/20)
df_full_train, df_test = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['Class']
)

# Train/Val split (60/20 of original)
df_train, df_val = train_test_split(
    df_full_train, test_size=0.25, random_state=42, stratify=df_full_train['Class']
)

print(f"전체 데이터: {len(df)}")
print(f"Train: {len(df_train)} ({len(df_train)/len(df)*100:.1f}%)")
print(f"Val: {len(df_val)} ({len(df_val)/len(df)*100:.1f}%)")
print(f"Test: {len(df_test)} ({len(df_test)/len(df)*100:.1f}%)")

In [ ]:
# 각 세트의 사기 비율 확인
print("사기 비율:")
print(f"Train: {df_train['Class'].mean():.4f}")
print(f"Val: {df_val['Class'].mean():.4f}")
print(f"Test: {df_test['Class'].mean():.4f}")

In [ ]:
# 데이터셋 리셋
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# Target 분리
y_train = df_train['Class'].values
y_val = df_val['Class'].values
y_test = df_test['Class'].values

# Feature만 남기기
X_train = df_train.drop('Class', axis=1)
X_val = df_val.drop('Class', axis=1)
X_test = df_test.drop('Class', axis=1)

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

## 4. Feature Engineering

현재는 모든 특성이 이미 숫자형이므로 추가적인 인코딩이 필요 없습니다.
하지만 나중에 Amount와 Time을 스케일링하거나 변환할 수 있습니다.

In [ ]:
# 현재는 원본 특성 그대로 사용
print("Features:")
print(X_train.columns.tolist())

## 5. Baseline Logistic Regression 모델

먼저 가장 기본적인 Logistic Regression 모델을 학습합니다.

In [ ]:
# Baseline 모델 학습
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

print("모델 학습 완료!")

In [ ]:
# Validation set 예측
y_pred_proba = model.predict_proba(X_val)[:, 1]  # 사기일 확률
y_pred = (y_pred_proba >= 0.5).astype(int)  # 기본 threshold 0.5

print(f"예측 완료: {len(y_pred)} samples")

## 6. 모델 평가 (불균형 데이터 고려)

불균형 데이터에서는 Accuracy만으로 평가하면 안 됩니다!
- **Precision**: 사기로 예측한 것 중 실제 사기 비율
- **Recall**: 실제 사기 중 찾아낸 비율
- **F1-Score**: Precision과 Recall의 조화평균
- **ROC-AUC**: 전체적인 분류 성능

In [ ]:
# 기본 평가 지표
print("=" * 50)
print("Baseline Model Performance (Threshold = 0.5)")
print("=" * 50)
print(f"Accuracy: {accuracy_score(y_val, y_pred):.4f}")
print(f"Precision: {precision_score(y_val, y_pred):.4f}")
print(f"Recall: {recall_score(y_val, y_pred):.4f}")
print(f"F1-Score: {f1_score(y_val, y_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_val, y_pred_proba):.4f}")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_val, y_pred)
print("\nConfusion Matrix:")
print(cm)

# 시각화
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

# TN, FP, FN, TP
tn, fp, fn, tp = cm.ravel()
print(f"\nTrue Negatives (정상을 정상으로): {tn}")
print(f"False Positives (정상을 사기로): {fp}")
print(f"False Negatives (사기를 정상으로): {fn}")
print(f"True Positives (사기를 사기로): {tp}")

In [ ]:
# Classification Report
print("\nClassification Report:")
print(classification_report(y_val, y_pred, target_names=['Normal', 'Fraud']))

In [ ]:
# ROC Curve
fpr, tpr, thresholds = roc_curve(y_val, y_pred_proba)
roc_auc = roc_auc_score(y_val, y_pred_proba)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'ROC Curve (AUC = {roc_auc:.4f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### Threshold 조정 실험

사기 탐지에서는 False Negative(사기를 놓치는 것)가 더 치명적일 수 있습니다.
Threshold를 낮춰서 Recall을 높일 수 있습니다.

In [ ]:
# 다양한 threshold 시도
thresholds_to_test = [0.3, 0.4, 0.5, 0.6, 0.7]

results = []
for threshold in thresholds_to_test:
    y_pred_thresh = (y_pred_proba >= threshold).astype(int)
    results.append({
        'threshold': threshold,
        'precision': precision_score(y_val, y_pred_thresh),
        'recall': recall_score(y_val, y_pred_thresh),
        'f1': f1_score(y_val, y_pred_thresh)
    })

results_df = pd.DataFrame(results)
print("\nThreshold별 성능:")
print(results_df)

In [ ]:
# 시각화
fig, ax = plt.subplots(figsize=(10, 5))
results_df.plot(x='threshold', y=['precision', 'recall', 'f1'], ax=ax, marker='o')
ax.set_xlabel('Threshold')
ax.set_ylabel('Score')
ax.set_title('Precision, Recall, F1-Score by Threshold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 7. 결과 분석 및 개선 방향

### 현재 모델의 특징
- 높은 Accuracy (대부분 정상 거래이므로)
- Precision과 Recall의 trade-off
- ROC-AUC로 전체적인 성능 평가

### 개선 방향
1. **데이터 불균형 처리**
   - SMOTE (Synthetic Minority Over-sampling)
   - Random Under-sampling
   - Class weight 조정

2. **Feature Engineering**
   - Amount, Time 스케일링
   - 새로운 파생 변수 생성

3. **모델 개선**
   - Regularization (C parameter 조정)
   - 다른 모델 시도 (Random Forest, XGBoost 등)

4. **Threshold 최적화**
   - 비즈니스 요구사항에 맞는 threshold 선택
   - Precision-Recall curve 분석

In [ ]:
# 모델 저장 (선택사항)
import pickle

# with open('../models/baseline_model.pkl', 'wb') as f:
#     pickle.dump(model, f)
# print("모델 저장 완료!")

## 다음 단계

1. `02-handling-imbalance.ipynb`: 불균형 데이터 처리 기법 적용
2. `03-feature-engineering.ipynb`: Feature 스케일링 및 새로운 특성 생성
3. `04-model-tuning.ipynb`: 하이퍼파라미터 튜닝 및 모델 개선
4. `05-deployment.ipynb`: 최종 모델 선택 및 배포 준비